In [31]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.vectorstores import Chroma
import os
import sqlite3
from langchain.schema import Document

In [32]:
tables_list_obj = ['assets_manage_asset', 'assets_manage_vehicle', 'core_problem', 'core_ambulance', 'core_bedsinventory', 'core_o2inventory', 'core_staffmember', 'maintain_maintenancetask', 'core_labor', 'core_attendance', 'core_taskassignment', 'core_task']

In [33]:
conn = sqlite3.connect('db.sqlite3')  # Ensure this path is correct
table_data = {}

In [34]:
cursor = conn.cursor()
    
for table_name in tables_list_obj:
    try:
        cursor.execute(f"SELECT * FROM {table_name};")
        rows = cursor.fetchall()
        
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        headers = [column[1] for column in columns]
        
        if rows:
            table_data[table_name] = {"headers": headers, "rows": rows}
    
    except sqlite3.Error as e:
        print(f"Error processing table {table_name}: {e}")


    docs = [
        f"Table: {table_name}\nHeaders: {', '.join(data['headers'])}\nRow: {', '.join(map(str, row))}"
        for table_name, data in table_data.items()
        for row in data['rows']
    ]

In [35]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text = '\n'.join(docs)
chunks = text_splitter.split_text(text)

documents = [Document(page_content=chunk) for chunk in chunks]

print(f"Generated {len(documents)} documents with content.")
documents

Generated 111 documents with content.


[Document(page_content='Table: assets_manage_asset\nHeaders: id, type, make, model, serial_number, condition, current_location, last_known_location, latitude, longitude, location_history, operational_status\nRow: 1, Excavator, Caterpillar, 320D, CAT320D123456, Good, Bhopal, Indore, 23.2599, 77.4126, [{"location": "Indore", "timestamp": "2024-01-10T12:00:00Z"}, {"location": "Bhopal", "timestamp": "2024-01-05T08:00:00Z"}], Operational\nTable: assets_manage_asset'),
 Document(page_content='Table: assets_manage_asset\nHeaders: id, type, make, model, serial_number, condition, current_location, last_known_location, latitude, longitude, location_history, operational_status\nRow: 2, Bulldozer, Komatsu, D65EX-18, KOMD6589123, Excellent, Indore, Gwalior, 22.7196, 75.8577, [{"location": "Gwalior", "timestamp": "2024-01-15T09:00:00Z"}, {"location": "Indore", "timestamp": "2023-12-30T14:00:00Z"}], Under Repair\nTable: assets_manage_asset'),
 Document(page_content='Table: assets_manage_asset\nHeader

In [36]:
text_chunks=documents
persist_directory = "db"
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectordb = Chroma.from_documents(documents=text_chunks, embedding=embedding, persist_directory=persist_directory)


I0000 00:00:1722399695.338976   13636 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [37]:
from dotenv import load_dotenv
load_dotenv()
google_gemini_api = os.getenv("GOOGLE_API_KEY")

In [38]:
retriever = vectordb.as_retriever()
llm_model = ChatGoogleGenerativeAI(model="models/gemini-1.5-pro", google_api_key=google_gemini_api)


I0000 00:00:1722399700.735028   13636 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722399700.736127   13636 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [39]:
system_prompt = (
    "You are an expert on the Municipal Corporation of Indore with access to detailed data. "
    "Answer questions based on this information, using concise and relevant details from the dataset. "
    "Context: {context}"
)


In [40]:
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [42]:
qa_chain=RetrievalQA.from_chain_type(
    llm=llm_model,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [43]:

class Memory:
    def __init__(self):
        self.history = []

    def add(self, entry):
        self.history.append(entry)
        if len(self.history) > 10:
            self.history.pop(0)

    def get_context(self):
        return " ".join(self.history)

memory = Memory()

In [44]:
def get_response(query):
    context = memory.get_context() + f" Recent query: {query}"
    print(f"Query: {query}\nContext: {context}")
    try:
        response = chain.invoke({"input": query, "context": context})
        answer = response.get('answer', '')
        print(f"Answer: {answer}")
        memory.add(f"User: {query}\nAI: {answer}")
        return answer
    except Exception as e:
        print(f"Error: {e}")
        return "Sorry, I couldn't process your request."

In [45]:
result=get_response("How many vehicles in good conditions")

Query: How many vehicles in good conditions
Context:  Recent query: How many vehicles in good conditions
Answer: There are **2 vehicles** in good condition. 

